In [5]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/post-postfilter.jsonl

In [1]:
from malaysian_sft import accept as accept_fn
import json
import random
from tqdm import tqdm
from glob import glob

In [2]:
system = [
    'always reply in malay',
    'sentiasa respond dalam bahasa melayu',
    'sentiasa jawab in malay',
    'awak adalah chatbot yang respond dalam bahasa melayu',
    'you are a chatbot that always respond in malay',
    'u always reply in malay',
    'awak sentiasa reply in malay',
]

In [3]:
!ls -lh postfilter.jsonl

ls: cannot access 'postfilter.jsonl': No such file or directory


In [10]:
filtered = []

with open('post-postfilter.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        
        q = l['instruction']

        if l['answer_ms'] is None:
            continue
            
        if len(set(l['answer_ms'].split())) < (len(set(l['answer'].split())) / 2):
            continue
            
        q = f'{random.choice(system)}\n\n{q.strip()}'
        
        d = {
            'prompt_input': None,
            'input': q,
            'output': l['answer_ms'].strip(),
        }
        filtered.append(d)

132352it [00:06, 20688.86it/s]


In [11]:
len(filtered)

132352

In [12]:
files = glob('/home/husein/ssd3/camel-ai/*.translated.jsonl')

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if 'message_1' not in l:
                continue
                
            if l['message_1'] is None or l['message_2_ms'] is None:
                continue
                
            q = f"{random.choice(system)}\n\n{l['message_1'].strip()}"
            
            d = {
                'prompt_input': None,
                'input': q,
                'output': l['message_2_ms'].strip(),
            }
            filtered.append(d)

50000it [00:00, 79179.13it/s] 
78577it [00:00, 563018.49it/s]
20000it [00:00, 80679.71it/s]
20000it [00:00, 85726.46it/s]
20000it [00:00, 80526.36it/s]


In [13]:
len(filtered)

232901

In [17]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian',
    'mistral',
    'terjemah',
    'translate'
]

In [18]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd2/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd2/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    q = r.get('content', '') or ''
                    current_text = q       
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                s = f"{role}: {current_text}"
                    
                inputs.append((s, r))

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 2:
                continue
            
            outputs = []
            for i in range(0, len(inputs), 2):
                
                try:
                    content = inputs[i][1]['content']
                    content_ms = inputs[i][1]['content_ms']

                    if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                        continue
                except:
                    pass
                
                try:
                    content = inputs[i + 1][1]['content']
                    content_ms = inputs[i + 1][1]['content_ms']

                    if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                        continue
                except:
                    pass
                
                outputs.extend([
                    inputs[i][0],
                    inputs[i + 1][0]
                ])
            
            if not len(outputs):
                continue
            outputs[0] = outputs[0].replace('<manusia>: ', f'<manusia>: {random.choice(system)}\n\n')

            data = '\n'.join(outputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            
            filtered.append(d)

60384it [00:14, 4198.54it/s]
48163it [00:09, 5237.80it/s]
57798it [00:11, 4849.60it/s]
103242it [01:05, 1576.12it/s]
135770it [01:57, 1154.05it/s]


In [19]:
len(filtered)

625080

In [27]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('prepared-force-malay.parquet')

In [28]:
!ls -lh prepared-force-malay.parquet

-rw-rw-r-- 1 husein husein 1000M Dis  24 20:41 prepared-force-malay.parquet


In [29]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-malay.parquet",
    path_in_repo='data/force_malay-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-malay.parquet:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/f5a1209b2824fefa5815292c084388c603b53ce2', commit_message='Upload data/force_malay-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='f5a1209b2824fefa5815292c084388c603b53ce2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)